In [1]:
%matplotlib inline
import numpy as np, pandas as pd, seaborn as sns, matplotlib.pyplot as plt
from pathlib import Path
from sklearn import datasets
plt.style.use("ggplot")
print("done")

done


Plan:
1) Fikse verdier i datasettet
2) Fjern variabler med varians over 50
3) Klipp ut eksempler med høye topper.
4) Bruk feature selection CHI2 og F_test med forskjellige antall variabler.

In [2]:
df = pd.read_csv('train.csv')
final_test = pd.read_csv('test.csv')
sampleSubmission = pd.read_csv('sample_submission.csv')
print("done")

done


In [3]:
df.head()

,id,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,...,f_66,f_67,f_68,f_69,f_70,f_71,f_72,f_73,f_74,target
0,161363,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,8.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,6
1,78028,16.0,0.0,1.0,1.0,6.0,2.0,2.0,2.0,14.0,...,0.0,41.0,3.0,0.0,0.0,0.0,2.0,1.0,1.0,7
2,35324,0.0,3.0,0.0,1.0,1.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,5
3,67966,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1
4,110279,3.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,NaN,...,0.0,0.0,1.0,5.0,4.0,0.0,0.0,0.0,1.0,5


In [4]:
X = df.drop(["target"], axis=1)
y = df.target

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

You can handle missing data/ NaN with imputation and interpolation, what i just found is interpolation is a type of estimation, a method of constructing new data points within the range of a discrete set of known data points while imputation is replacing the missing data of the mean of the column.

In [6]:
from sklearn.ensemble import VotingClassifier

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB

rf = RandomForestClassifier(random_state=42, n_jobs=-1)
gb = GradientBoostingClassifier(random_state=42)
svc = SVC(random_state=42)
gnb = GaussianNB()

models = [('rf', rf), 
          ('gnb', gnb)]

ensemble = VotingClassifier(models)

('gb', gb),
('svc', svc),

In [7]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import make_pipeline

In [8]:
pipeline = make_pipeline(SimpleImputer(), 
                         MinMaxScaler(), 
                         ensemble)
pipeline

Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('minmaxscaler', MinMaxScaler()),
                ('votingclassifier',
                 VotingClassifier(estimators=[('rf',
                                               RandomForestClassifier(n_jobs=-1,
                                                                      random_state=42)),
                                              ('gnb', GaussianNB())]))])

In [9]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('minmaxscaler', MinMaxScaler()),
                ('votingclassifier',
                 VotingClassifier(estimators=[('rf',
                                               RandomForestClassifier(n_jobs=-1,
                                                                      random_state=42)),
                                              ('gnb', GaussianNB())]))])

In [10]:
from sklearn.model_selection import cross_val_score

In [11]:
cross_val_score(pipeline, X_train, y_train, cv=3, scoring='accuracy')

array([0.16632   , 0.16973333, 0.16605333])

In [12]:
from sklearn.preprocessing import StandardScaler

In [15]:
for scaler in [MinMaxScaler(), StandardScaler()]:
    pipeline = make_pipeline(SimpleImputer(), scaler, ensemble)
    
    scores = cross_val_score(pipeline, X_train, y_train, cv=3, scoring='accuracy')
    print(f"Cross val scores for {scaler}: {scores}")
    print(f"Mean: {np.mean(scores)}\n")

Cross val scores for MinMaxScaler(): [0.16632    0.16973333 0.16605333]
Mean: 0.1673688888888889

Cross val scores for StandardScaler(): [0.16626667 0.16968    0.16602667]
Mean: 0.16732444444444447



In [ ]:
submission = pd.DataFrame({'id': test['id'], 'target': target})
submission.head()

In [ ]:
submission.to_csv('pipeline_simple.csv', index=False)

In [ ]:
from sklearn.feature_selection import mutual_info_classif
# determine the mutual information
mutual_info1 = mutual_info_classif(X_train_initial.fillna(0), y_train_initial) # trenger ikke fillna lenger
mutual_info1

Uten interpolering (fillna=0):
f_18    0.042137
f_37    0.041022
f_43    0.037857
f_40    0.034660
f_12    0.034154
f_72    0.031311
f_14    0.031282
f_53    0.031225
f_3     0.030770
f_2     0.029875
f_65    0.028901
f_28    0.028273
f_31    0.028162
f_1     0.023683
f_8     0.022663
f_56    0.021963
f_16    0.021600
f_50    0.021356
f_21    0.021214
f_44    0.019598
f_45    0.017552
f_20    0.015884
f_13    0.015006
f_23    0.014988
f_39    0.014931
f_70    0.014799
f_7     0.014777
f_4     0.014397
f_9     0.014185
f_48    0.013002
f_35    0.012888
f_52    0.012756
f_19    0.012713
f_55    0.012549
f_67    0.012505
f_38    0.012472
f_46    0.012380
f_24    0.012324
f_29    0.012053
f_66    0.011818
f_69    0.011174
f_25    0.011089
f_60    0.010973
f_11    0.010766
f_54    0.010556
f_51    0.010498
f_73    0.010435
f_61    0.010419
f_30    0.010275
f_63    0.009903
f_15    0.009605
f_0     0.008954
f_62    0.008832
f_58    0.008633
f_41    0.008576
f_5     0.007997
f_22    0.007865
f_57    0.006573
f_59    0.006510
f_64    0.006493
f_10    0.006238
f_32    0.006193
f_49    0.006045
f_71    0.005949
f_68    0.005535
f_47    0.005516
f_34    0.005200
f_26    0.003571
f_27    0.003545
f_42    0.002903
f_33    0.002703
f_17    0.002457
f_74    0.000713
f_36    0.000000
f_6     0.000000
id      0.000000
dtype: float64

MED interpolering
f_12    0.042418
f_43    0.039170
f_37    0.038279
f_18    0.037486
f_3     0.033675
f_40    0.032602
f_53    0.030294
f_65    0.029889
f_28    0.029495
f_72    0.029124
f_2     0.028865
f_14    0.028207
f_31    0.028141
f_1     0.024191
f_16    0.024024
f_44    0.022703
f_56    0.022226
f_50    0.021610
f_8     0.020229
f_21    0.019420
f_9     0.019318
f_20    0.017315
f_67    0.016867
f_23    0.016712
f_35    0.016097
f_48    0.015825
f_38    0.015787
f_46    0.013802
f_45    0.013741
f_52    0.013390
f_60    0.013352
f_41    0.013239
f_4     0.013227
f_24    0.012848
f_25    0.012364
f_70    0.012034
f_69    0.011748
f_59    0.011730
f_62    0.011351
f_13    0.011233
f_66    0.011221
f_7     0.011100
f_55    0.011004
f_19    0.010612
f_29    0.010602
f_51    0.010298
f_39    0.010243
f_30    0.009758
f_15    0.009304
f_63    0.009089
f_32    0.008619
f_58    0.008456
f_54    0.007683
f_0     0.007531
f_49    0.007453
f_61    0.007391
f_73    0.007064
f_22    0.006907
f_68    0.006840
f_5     0.006454
f_27    0.006199
f_10    0.005728
f_64    0.005298
f_26    0.005121
f_33    0.005023
f_42    0.004549
f_57    0.004239
f_34    0.004181
f_17    0.004178
f_71    0.003958
f_47    0.003176
f_11    0.002550
f_74    0.002061
f_6     0.001887
f_36    0.001475
id      0.000000
dtype: float64

In [ ]:
mutual_info1 = pd.Series(mutual_info1)
mutual_info1.index = X_train_initial.columns
mutual_info1.sort_values(ascending=False)

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2

selector_class = SelectKBest(mutual_info_classif, k=47)
selector_class.fit(X_train_initial.fillna(0), y_train_initial)

# Get columns for new dataframe
cols = selector_class.get_support(indices=True)
features_selected = X_train_initial.iloc[:,cols]


In [ ]:
features_selected.keys()

In [ ]:
X = df[['f_1', 'f_2', 'f_3', 'f_4', 'f_7', 'f_8', 'f_9', 'f_12', 'f_13', 'f_14',
       'f_16', 'f_18', 'f_19', 'f_20', 'f_21', 'f_23', 'f_24', 'f_25', 'f_28',
       'f_29', 'f_30', 'f_31', 'f_35', 'f_37', 'f_38', 'f_39', 'f_40', 'f_41',
       'f_43', 'f_44', 'f_45', 'f_46', 'f_48', 'f_50', 'f_52', 'f_53', 'f_56',
       'f_60', 'f_61', 'f_62', 'f_65', 'f_66', 'f_67', 'f_69', 'f_70', 'f_72',
       'f_73']].fillna(0)
y = df.target

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state=42)
print("set is split")

In [ ]:
# standardize
from sklearn.preprocessing import StandardScaler

std = StandardScaler()
X_train_std = std.fit_transform(X_train)
X_test_std = std.transform(X_test)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
k_range = range(5,45,10)
scores = []

for k in k_range:
    knn = KNeighborsClassifier(n_neighbors = k)
    knn.fit(X_train_std, y_train)
    knn_y_test_pred = knn.predict(X_test_std)
    scores.append(metrics.accuracy_score(y_test, knn_y_test_pred))

plt.plot(k_range, scores)
plt.xlabel("Value of k for KNN")
plt.ylabel("Testing accuracy")

In [ ]:
scores[-1]

In [ ]:
#Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(random_state=42)
rf_clf.fit(X_train, y_train)

rf_clf_y_train_pred = rf_clf.predict(X_train_std) #predict on train set
rf_clf_y_test_pred = rf_clf.predict(X_test_std) # predict on test set

print("Train accuracy: ", metrics.accuracy_score(y_train, rf_clf_y_train_pred))
print("Test accuracy: ", metrics.accuracy_score(y_test, rf_clf_y_test_pred))

In [ ]:
df_new = df.interpolate(method ='linear', limit_direction ='forward')
df_new = df_new.interpolate(method ='linear', limit_direction ='backward', limit = 1)

In [ ]:
df_new.head()

In [ ]:
# Variable discarded due to high variance > 50
df_new_var = df_new.drop(["id", "f_3", "f_15", "f_19", "f_25", "f_28", "f_32", "f_43", "f_46", "f_54", "f_55", "f_62", "f_67", "f_73"], axis=1)

In [ ]:
# Filter by variance. Remove all above 50.
pd.set_option('display.max_rows', 500)
df_new_var.var() > 50

# variance is the expectation of the squared deviation of a random variable from its population mean or sample mean.

In [ ]:
df_new_var.shape

In [ ]:
# Hvor mange målinger er på max.
def num_of_max(df, key):
    return (df[key] == max(df[key])).sum()

def num_of_min(df, key):
    return (df[key] == min(df[key])).sum()

for i in df_new_var.keys():
    print(i, ": Max:", num_of_max(df_new_var, i), " Min:", num_of_min(df_new_var, i))
    
# Alle variabler har så få på max at vi antar det er feilmålinger.

In [ ]:
df_new_var_no_max = df_new_var

In [ ]:
for i in df_new_var_no_max.keys():
    if i != "target":
        if (df_new_var_no_max[i] == max(df_new_var_no_max[i])).sum() < 100:
            df_new_var_no_max = df_new_var_no_max.drop(df_new_var_no_max[df_new_var_no_max[i]==df_new_var_no_max[i].max()].index.tolist())

for i in df_new_var_no_max.keys():
    print(i, ": Max:", num_of_max(df_new_var_no_max, i), " Min:", num_of_min(df_new_var_no_max, i))
print(df_new_var_no_max.shape)
# Prøver 2 kjøringer først.

In [ ]:
# Split til trening og test set
y = df_new_var_no_max.target
X = df_new_var_no_max.drop(["target"], axis=1)
X.head()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state=42)
print("set is split")

In [ ]:
from sklearn.feature_selection import mutual_info_classif
# determine the mutual information
mutual_info = mutual_info_classif(X_train, y_train)
mutual_info

In [ ]:
mutual_info = pd.Series(mutual_info)
mutual_info.index = X_train.columns
mutual_info.sort_values(ascending=False)

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2

selector_class = SelectKBest(chi2, k=5)
selector_class.fit(X_train, y_train)

# Get columns for new dataframe
cols = selector_class.get_support(indices=True)
X_selected_class_train = X_train.iloc[:,cols]
X_selected_class_test = X_test.iloc[:,cols]

In [ ]:
X_selected_class_train.head()

In [ ]:
#Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(random_state=42)
rf_clf.fit(X_selected_class_train, y_train)

rf_clf_y_train_pred = rf_clf.predict(X_selected_class_train) #predict on train set
rf_clf_y_test_pred = rf_clf.predict(X_selected_class_test) # predict on test set

print("Train accuracy: ", metrics.accuracy_score(y_train, rf_clf_y_train_pred))
print("Test accuracy: ", metrics.accuracy_score(y_test, rf_clf_y_test_pred))

# Alle features:
# Train accuracy:  0.9989448960493238
# Test accuracy:  0.3455631399317406

# 50 features:

# Train accuracy:  0.9894306108593134
# Test accuracy:  0.3392876802818452

# 20 features
#Train accuracy:  0.904003890122392
#Test accuracy:  0.3217274028404712





In [16]:
from math import comb


74 + 74 + comb(74,2)



2849

In [ ]:
#EVT F_TEST
sel_f = SelectKBest(f_classif, k=4)
X_train_f = sel_f.fit_transform(X_train, y_train)

print(sel_f.get_support())
print(sel_f.get_params())